In [1]:
import os
import tempfile

import coiled
from archetypal.idfclass import IDF
from archetypal.idfclass.sql import Sql
from pydantic import AnyUrl

from epengine.models.configs import SimulationSpec
from epengine.utils.results import postprocess, serialize_df_dict


In [ ]:
AWS_ACCOUNT_ID = os.environ["AWS_ACCOUNT_ID"]
AWS_REGION = os.environ["AWS_REGION"]

worker_image_name = "ml-for-bem-coiledworker"


@coiled.function(
    region="us-east-1",
    vm_type=["t3.medium", "t3.large"],
    container=f"{AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com/{worker_image_name}:latest",
)
def run_simulation(data: dict[str, str], ix: int):
    """Run a simulation using the provided data and return the results.

    Args:
        data (dict[str, str]): The data to run the simulation with.
        ix (int): The index of the simulation.

    Returns:
        dict[str, pd.DataFrame]: The results of the simulation
    """
    spec = SimulationSpec.model_validate(data)
    with tempfile.TemporaryDirectory() as tmpdir:
        idf = IDF(
            spec.idf_path,  # pyright: ignore [reportArgumentType]
            epw=spec.epw_path,
            output_directory=tmpdir,
        )  # pyright: ignore [reportArgumentType]
        idf.simulate()
        sql = Sql(idf.sql_file)
        index_data = spec.model_dump(mode="json", exclude_none=True)
        index_data["spawn_index"] = ix
        # TODO: pull in spawn index
        dfs = postprocess(
            sql,
            index_data=index_data,
            tabular_lookups=[("AnnualBuildingUtilityPerformanceSummary", "End Uses")],
        )
    dfs = serialize_df_dict(dfs)
    return dfs

In [8]:
AWS_BUCKET = "ml-for-bem"
spec = SimulationSpec(
    experiment_id="coiled-test",
    idf_uri=AnyUrl(f"s3://{AWS_BUCKET}/hatchet/insomnia-test/idf/Office_totalBaseline.idf"),
    epw_uri=AnyUrl(f"s3://{AWS_BUCKET}/hatchet/insomnia-test/epw/ARG_Buenos.Aires.875760_IWEC.epw"),
    ddy_uri=None,
)
arg = spec.model_dump(mode="json")
results = run_simulation.map([arg for _ in range(10)], range(10), errors="skip")
results = list(results)

In [9]:
import pandas as pd

pd.concat([
    pd.DataFrame.from_dict(result["AnnualBuildingUtilityPerformanceSummary_End_Uses"], orient="tight")
    for result in list(results)
])

ColumnName                                                                                                                      Electricity  \
Units                                                                                                                                    GJ   
RowName                                                                                                                             Cooling   
experiment_id idf_uri                                            epw_uri                                            spawn_index               
coiled-test   s3://ml-for-bem/hatchet/insomnia-test/idf/Offic... s3://ml-for-bem/hatchet/insomnia-test/epw/ARG_B... 0                   0.0   
                                                                                                                    1                   0.0   
                                                                                                                    2                   0.0   
                                                                                                                    3                   0.0   
                                                                                                                    4                   0.0   
                                                                                                                    5                   0.0   
                                                                                                                    6                   0.0   
                                                                                                                    7                   0.0   
                                                                                                                    8                   0.0   
                                                                                                                    9                   0.0   

ColumnName                                                                                                                                          \
Units                                                                                                                                                
RowName                                                                                                                         Exterior Equipment   
experiment_id idf_uri                                            epw_uri                                            spawn_index                      
coiled-test   s3://ml-for-bem/hatchet/insomnia-test/idf/Offic... s3://ml-for-bem/hatchet/insomnia-test/epw/ARG_B... 0                          0.0   
                                                                                                                    1                          0.0   
                                                                                                                    2                          0.0   
                                                                                                                    3                          0.0   
                                                                                                                    4                          0.0   
                                                                                                                    5                          0.0   
                                                                                                                    6                          0.0   
                                                                                                                    7                          0.0   
                                                                                                                    8                          0.0   
                                               